# Demo of Google's Gemma Model - Answering Common Python Questions

By Nicole Michaud
04/08/2024

[Check out my Github here](http://github.com/nicolemichaud03)

[Connect with me on LinkedIn](http://linkedin.com/in/nicole-michaud2)

<div style="text-align: center;">
<img align="center" src = "https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdWSrhXoP4nHcjFcmbTNjhKcd3LRLcIiL2uEp4_8ilX7h_zsMu_muQlLP52eEgxvq4ejAQKy0TQKNaFC07O4o9imxqDDKF8hgaLU-iYfwmcPYGpm64psp1WHyaJOZQPImAhCDpYtc4nWEvbM3hSERTA50n08rIhftkP0rK1ai9uB-o3nWx0TQMRWt1leQ/w1200-h630-p-k-no-nu/Keras-Gemma-GfD.png" width="700">
</div> 


#### Purpose:
The goal of this notebook is to demonstrate how to use the Gemma LLM to answer common questions about the Python programming language. Gemma was trained on web documents, code, and mathematics, so these question and answer pairs about the Python programming language should work great with it! The data used for this task was sourced from FAQs on Python's website.

#### What is Gemma?
Gemma is a lightweight, open-source family of models from Google, built from the same research and technology used to create their Gemini models
Gemma models come in two different parameter size options: 2B and 7B. The 2B model is more lightweight and will perform just fine for this demo.

Let's get started!

## Data Preparation

First, we need to import the necessary libraries for data processing and so that we are able to run this notebook on Kaggle with the Gemma model:

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

In [ ]:
import keras
import keras_nlp

Keras can be used to easily establish the basic architecture for the model to run on using either TensorFlow, JAX, or PyTorch. I am going to be using TensorFlow in this demo.

In [ ]:
#establish a backend
os.environ["KERAS_BACKEND"] = "tensorflow"  


The dataset that is going to be used was created from <a href="https://docs.python.org/3/faq/general.html">Python's FAQs page</a> and includes the common questions, their answers, and what category they fall under.

In [ ]:
#import the dataset
import pandas as pd
documents = pd.read_csv("/kaggle/input/python-faq-qa/python_faqs.csv")


### Data Exploration

In [ ]:
documents.head()

In [ ]:
documents.info()

The dataset contains 178 question-answer pairs.

In [ ]:
documents['Category'].unique()

The FAQ questions each fall into one of eight different categories.

## Modeling

For this task I will be using the Gemma_2b model architecture, because I am running it on a personal computer that may not be able to support the 7b architecture. I will be using the GemmaCausalLM model, an end-to-end model for causal language modeling, configured with a preprocessor layer (GemmaCausalLMPreprocessor). By using a preprocessor, the string inputs are automatically preprocessed/tokenized, as opposed to having to do these steps prior to loading the model. The <a href="https://keras.io/api/keras_nlp/models/gemma/https://keras.io/api/keras_nlp/models/gemma/" >Keras 3 API documentation</a> has more details on the different steps and options for Gemma's pretrained models.

#### Loading the Model:

In [ ]:
# Load a preprocessor layer from a preset.
preprocessor = keras_nlp.models.GemmaCausalLMPreprocessor.from_preset(
    "gemma_2b_en",
)

In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en", preprocessor=preprocessor)
gemma_lm.summary()

#### Generating prompts:

Even before fine-tuning the model, we can use it to generate answers. First, we have to generate some sample prompts to be able to see how the model is doing at answering common Python questions, before we fine-tune it.First, we have to generate some sample prompts to be able to see how the model is doing at answering common Python questions, before we fine-tune it.

In [ ]:
#Need to format each question-answer pair with a template that joins together and labels the Question, Answer and Category of each
#to be able to be interpreted by the model
from tqdm.notebook import tqdm
tqdm.pandas() 
template = "\n\nCategory:\nkaggle-{Category}\n\nQuestion:\n{Question}\n\nAnswer:\n{Answer}"
documents["prompt"] = documents.progress_apply(lambda row: template.format(Category=row.Category,
                                                             Question=row.Question,
                                                             Answer=row.Answer), axis=1)
documents = documents.prompt.tolist()

For these prompts I selected 3 random questions from 3 different categories of FAQs, to get an idea of how the model performs on different types of questions.


In [ ]:
# Question from "general" category

prompt = template.format(
    Question="What is Python?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

The specific question used in the prompt was given a pretty decent answer, but the other questions generated are not related to Python (they are about another programming language, R).

In [ ]:
# Question from "extending" category

prompt = template.format(
    Question="How do I catch the output from PyErr_Print() (or anything that prints to stdout/stderr)?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

The answer provided to this question is a bit confusing, it has text from someone named John, which seems a little out of place.

In [ ]:
# Question from "library" category

prompt = template.format(
    Question="How do I program using threads?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

The above answers look okay, but some parts of them do not completely make sense for what is being asked. Let's see if we can improve them by fine-tuning the model.

### Fine-tuning with LoRA (Low-Rank Adaptation):

Gemma models and pre-trained, however, a process called fine-tuning can be used to further train them on a particular dataset!

According to the documentation, **LoRA** is a form of PEFT, or parameter-efficient fine-tuning. This is because it works by reducing down the number of trainable parameters and freezing model weights, causing the model to run faster and be more memory-efficient, while maintaining high-quality outputs! 
You can learn more about the specifics of how LoRA works to fine-tune Gemma models <a href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb#scrollTo=lSGRSsRPgkzKhttps://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemma/docs/lora_tuning.ipynb#scrollTo=lSGRSsRPgkzK">here!</a>


There are other options for tuning Gemma models, but for this demo, LoRA fine-tuning via KerasNLP is a good choice because it is known to be efficient and effective.

When enabling LoRA, the rank must be specified. The rank is related to the number of trainable parameters to be used, and a higher rank indicates that more detailed changes can be made to the model. However, the higher the rank, the more computationally expensive it will be. Therefore, it is best to start with a relatively low rank. Here, we use rank=4.

In [ ]:
# Enable LoRA freeze weights on the backbone, while allowing Lora to tune the query and value (question and answer) layers

gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

The total params are a little more than before enabling Lora, now 2,507,536,382 from 2,506,172,416. However, the trainable params are only 1,363,968 out of those whereas all of the params were trainable previously. This is because LoRa reduces trainable parameters in order to reduce the time and memory required for the model to run.

In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
#The Adam optimizer is used in the Gemma documentation, but other optimizers can be used as well
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(documents, epochs=1, batch_size=1)

I used the same hyperparameter values as the documentation <a href = "https://ai.google.dev/gemma/docs/lora_tuning#lora_fine-tuninghttps://ai.google.dev/gemma/docs/lora_tuning#lora_fine-tuning">here</a>. Some other options for fine-tuning parameters include using a different optimizer, testing out different learning rates and weight decays, and changing the number of epochs and batch size. With respect to the loss, the model is adjusting its weights with each step. How fast or slow it does this adjustment is dictated by the **learning rate**. **Weight decay** is a regularization technique that will cause the weights to exponentially decay (via the specified learning rate) until they equal zero.

Generally, models perform better with multiple epochs, as the model is given more time to understand the trends within the data. Also, a larger batch size (the group of data points that the model is being fit on) can improve performance, but it can also cause overfitting (model is over-trained on the training data and therefore does not know how to respond to test/unseen data). 

As you can see, the above model took a long time to run. This runtime will only increase as you increase the number of epochs and the batch size. Therefore you must keep in mind the tradeoff between optimal model performance and runtime/memory usage.

### Inference after fine-tuning

Let's try out the same sample questions used before to see how the answers have changed after fine-tuning.

In [ ]:
prompt = template.format(
    Question="What is Python?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    Question= "How do I catch the output from PyErr_Print() (or anything that prints to stdout/stderr)?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
prompt = template.format(
    Question= "How do I program using threads?",
    Answer="",
    Category=""
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

The generated answers for our sample questions are looking much better and making more sense after-fine tuning!

The next steps would be to continue tying out different hyperparameter values for fine-tuning to see how much we can improve the accuracy and the answers we get!

### Conclusion

Google's new Gemma LLM models are lightweight and user friendly as they can be used with familiar APIs such as Keras. In this demo, I show you how to use Gemma with KerasNLP to answer common questions about the programming language Python! 